## 📚 Prerequisites

Before executing this notebook, make sure you have properly set up your Azure Services, created your Conda environment, and configured your environment variables as per the instructions provided in the [README.md](README.md) file.

## 📋 Table of Contents

This notebook guides you through the following sections:

1. [**Create an Azure Cognitive Search Index**](#create-an-azure-cognitive-search-index): This index will store the content from a document hosted on SharePoint Online.

2. [**Convert PDF Documents into Images using `OCRHelper`**](#convert-pdf-documents-into-images-using-ocrhelper): This helper manages the connection to an Azure Blob Storage container and assists in converting PDF files into images, which can later be consumed by the OCR GPT-4 Vision model.

3. [**Using GPT-4 Vision for OCR**]: After initializing the `GPT4VisionManager`, we use it to perform Optical Character Recognition (OCR) on an image. The GPT-4 Vision model is capable of recognizing and extracting text from images, which can be useful in a variety of applications such as document analysis, data extraction, and more.


4. [**Ingest into Azure AI Search Index**](#ingest-index): The extracted content and metadata are ingested into the Azure AI Search Index for easy retrieval and search.

For more details, refer to the following resources:
- [Quickstart: Register an app with the Azure AD v2.0 endpoint](https://learn.microsoft.com/en-us/azure/active-directory/develop/console-app-quickstart?pivots=devlang-python)

## Getting Started

#### Configure Environment Variables 

Before running this notebook, you must configure certain environment variables. We will now use environment variables to store our configuration. This is a more secure practice as it prevents sensitive data from being accidentally committed and pushed to version control systems.

Create a `.env` file in your project root (use the provided `.env.sample` as a template) and add the following variables:

```env
# Azure AI Search Service Configuration
AZURE_AI_SEARCH_SERVICE_ENDPOINT="[Your Azure Search Service Endpoint]"
AZURE_SEARCH_ADMIN_KEY="[Your Azure Search Index Name]"

#Azure Open API Configuration
AZURE_OPENAI_API_KEY='[Your OpenAI API Key]'
AZURE_OPENAI_ENDPOINT='[Your OpenAI Endpoint]'
AZURE_OPENAI_API_VERSION='[Your Azure OpenAI API Version]'

#Azure Open API Configuration
AZURE_STORAGE_CONNECTION_STRING='[Your Azure Storage Connection String]'
```

Replace the placeholders (e.g., [Your Azure Search Service Endpoint]) with your actual values.

- `AZURE_AI_SEARCH_SERVICE_ENDPOINT` and `AZURE_SEARCH_ADMIN_KEY` are used to configure the Azure AI Search service.
- `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT`, and `AZURE_OPENAI_API_VERSION` are used to configure the Azure OpenAI service.
- `AZURE_STORAGE_CONNECTION_STRING` is used to configure the Azure Storage service.
```

> 📌 **Note**
> Remember not to commit the .env file to your version control system. Add it to your .gitignore file to prevent it from being tracked.

#### Setting Up Conda Environment and Configuring VSCode for Jupyter Notebooks (Optional)

Follow these steps to create a Conda environment and set up your VSCode for running Jupyter Notebooks:

##### Create Conda Environment from the Repository

> Instructions for Windows users: 

1. **Create the Conda Environment**:
   - In your terminal or command line, navigate to the repository directory.
   - Execute the following command to create the Conda environment using the `environment.yml` file:
     ```bash
     conda env create -f environment.yml
     ```
   - This command creates a Conda environment as defined in `environment.yml`.

2. **Activating the Environment**:
   - After creation, activate the new Conda environment by using:
     ```bash
     conda activate sharepoint-indexing
     ```

> Instructions for Linux users (or Windows users with WSL or other linux setup): 

1. **Use `make` to Create the Conda Environment**:
   - In your terminal or command line, navigate to the repository directory and look at the Makefile.
   - Execute the `make` command specified below to create the Conda environment using the `environment.yml` file:
     ```bash
     make create_conda_env
     ```

2. **Activating the Environment**:
   - After creation, activate the new Conda environment by using:
     ```bash
     conda activate sharepoint-indexing
     ```

##### Configure VSCode for Jupyter Notebooks

1. **Install Required Extensions**:
   - Download and install the `Python` and `Jupyter` extensions for VSCode. These extensions provide support for running and editing Jupyter Notebooks within VSCode.

2. **Open the Notebook**:
   - Open the Jupyter Notebook file (`01-indexing-content.ipynb`) in VSCode.

3. **Attach Kernel to VSCode**:
   - After creating the Conda environment, it should be available in the kernel selection dropdown. This dropdown is located in the top-right corner of the VSCode interface.
   - Select your newly created environment (`sharepoint-indexing`) from the dropdown. This sets it as the kernel for running your Jupyter Notebooks.

4. **Run the Notebook**:
   - Once the kernel is attached, you can run the notebook by clicking on the "Run All" button in the top menu, or by running each cell individually.


By following these steps, you'll establish a dedicated Conda environment for your project and configure VSCode to run Jupyter Notebooks efficiently. This environment will include all the necessary dependencies specified in your `environment.yml` file. If you wish to add more packages or change versions, please use `pip install` in a notebook cell or in the terminal after activating the environment, and then restart the kernel. The changes should be automatically applied after the session restarts.

## Create an Azure Cognitive Search Index

In [11]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SearchIndex,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SearchField,
    VectorSearch,
    SemanticSearch,
    HnswAlgorithmConfiguration,
    HnswParameters,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    ExhaustiveKnnParameters,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SearchField,
    VectorSearch,
    HnswParameters,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)

# Define the target directory (change yours)
target_directory = r"C:\Users\pablosal\Desktop\gbbai-azure-ai-document-intelligence"

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-azure-ai-document-intelligence


In [12]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set the service endpoint and API key from the environment
# Create an SDK client
endpoint = os.environ["AZURE_AI_SEARCH_SERVICE_ENDPOINT"]
search_client = SearchClient(
    endpoint=endpoint,
    index_name=os.environ["SEARCH_INDEX_NAME"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]),
)

admin_client = SearchIndexClient(
    endpoint=endpoint,
    index_name=os.environ["SEARCH_INDEX_NAME"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]),
)

In [3]:
# Delete the index if it exists
try:
    result = admin_client.delete_index(os.environ["SEARCH_INDEX_NAME"])
    print("Index", os.environ["SEARCH_INDEX_NAME"], "Deleted")
except Exception as ex:
    print(ex)

Index azure-ocr-index Deleted


In [4]:
fields = [
    SimpleField(
        name="id",
        type=SearchFieldDataType.String,
        key=True,
        sortable=True,
        filterable=True,
        facetable=True,
    ),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="summary", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchableField(name="category", type=SearchFieldDataType.String, filterable=True),
    SearchField(
        name="summaryVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1536,
        vector_search_profile_name="myHnswProfile",
    ),
    SearchField(
        name="contentVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1536,
        vector_search_profile_name="myHnswProfile",
    ),
]

In [5]:
# Configure the vector search configuration
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw",
            kind=VectorSearchAlgorithmKind.HNSW,
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric=VectorSearchAlgorithmMetric.COSINE,
            ),
        ),
        ExhaustiveKnnAlgorithmConfiguration(
            name="myExhaustiveKnn",
            kind=VectorSearchAlgorithmKind.EXHAUSTIVE_KNN,
            parameters=ExhaustiveKnnParameters(
                metric=VectorSearchAlgorithmMetric.COSINE
            ),
        ),
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        ),
        VectorSearchProfile(
            name="myExhaustiveKnnProfile",
            algorithm_configuration_name="myExhaustiveKnn",
        ),
    ],
)

In [6]:
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        keywords_fields=[SemanticField(field_name="category")],
        content_fields=[SemanticField(field_name="content")],
    ),
)
# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

In [7]:
index = SearchIndex(
    name=os.environ["SEARCH_INDEX_NAME"],
    fields=fields,
    vector_search=vector_search,
    semantic_search=semantic_search,
)

try:
    result = admin_client.create_or_update_index(index)
    print("Index", result.name, "created")
except Exception as ex:
    print(ex)

Index azure-ocr-index created


## Convert PDF Documents into Images using `OCRHelper`

This section of code is using the `OCRHelper` class from the `ocr_data_extractor` module to extract images from a PDF file stored in an Azure Blob Storage container.

Here's a step-by-step explanation:

1. `ocr_data_extractor_helper = OCRHelper(container_name="ocrtest")`: This line initializes an instance of the `OCRHelper` class, which is designed to interact with a specific Azure Blob Storage container. In this case, the container is named "ocrtest".

2. `INPUT_PATH` and `OUTPUT_PATH` are defined. `INPUT_PATH` is the URL of the PDF file in the Azure Blob Storage container, and `OUTPUT_PATH` is the local directory where the extracted images will be saved.

3. `ocr_data_extractor_helper.extract_images_from_pdf(input_path=INPUT_PATH, output_path=OUTPUT_PATH)`: This line calls the `extract_images_from_pdf` method of the `OCRHelper` instance. This method downloads the PDF file from the `INPUT_PATH`, converts each page of the PDF into an image, and saves the images to the `OUTPUT_PATH`.

In [13]:
from src.extractors.ocr_data_extractor import OCRHelper

ocr_data_extractor_helper = OCRHelper(container_name="ocrtest")

2024-01-04 18:26:02,080 - micro - MainProcess - INFO     Initialized AzureBlobManager with container ocrtest (blob_data_extractor.py:__init__:50)


In [18]:
# Replace with the URL of your PDF file in Azure Blob Storage
INPUT_PATH = "https://testeastusdev001.blob.core.windows.net/ocrtest/"

# Replace with the path to your local directory where the images will be saved
OUTPUT_PATH = "C:\\Users\\pablosal\\Desktop\\gbbai-azure-ai-document-intelligence\\notebooks\\dev\\images"

ocr_data_extractor_helper.extract_images_from_pdf(
    input_path=INPUT_PATH, output_path=OUTPUT_PATH
)

2024-01-04 18:30:20,695 - micro - MainProcess - INFO     Input path is a URL: https://testeastusdev001.blob.core.windows.net/ocrtest/ (ocr_data_extractor.py:extract_images_from_pdf:45)
2024-01-04 18:30:20,830 - micro - MainProcess - INFO     instruction-manual-fieldvue-dvc6200-hw2-digital-valve-controller-en-123052.pdf (blob_data_extractor.py:download_files_to_folder:132)
2024-01-04 18:30:22,719 - micro - MainProcess - INFO     Downloaded instruction-manual-fieldvue-dvc6200-hw2-digital-valve-controller-en-123052.pdf to C:\Users\pablosal\AppData\Local\Temp\tmp0udetmqb\instruction-manual-fieldvue-dvc6200-hw2-digital-valve-controller-en-123052.pdf (blob_data_extractor.py:download_files_to_folder:139)
2024-01-04 18:30:22,719 - micro - MainProcess - INFO     instruction-manual-fisher-ewd-ews-ewt-valves-through-nps-12x8-en-124788.pdf (blob_data_extractor.py:download_files_to_folder:132)
2024-01-04 18:30:23,456 - micro - MainProcess - INFO     Downloaded instruction-manual-fisher-ewd-ews-ewt-

## Utilizing GPT-4 Vision for OCR and Information Extraction

In this section, we initialize the `GPT4VisionManager` and use it to perform Optical Character Recognition (OCR) on an image. The `GPT4VisionManager` is a class from the `src.ocr.transformer` module that manages the interaction with the GPT-4 Vision model.

The `call_gpt4v_image` method of the `GPT4VisionManager` is used to perform OCR on the image at `image_file_path`. This method takes several parameters:

- `image_file_path`: The path to the image file.
- `system_instruction`: A high-level instruction that sets the role of the AI.
- `user_instruction`: A specific instruction that tells the AI what to do.
- `ocr`: If set to `True`, OCR is enabled.
- `use_vision_api`: If set to `True`, the Vision API is used.
- `display_image`: If set to `True`, the image is displayed.
- `max_tokens`: Limits the number of tokens in the output.
- `seed`: Sets the seed for reproducibility.

The `system_instruction` and `user_instruction` parameters are used to guide the AI model's behavior. They are essentially the instructions that you give to the AI model.

`system_instruction` is a high-level instruction that sets the role of the AI. In your case, `sys_message` is set to "You are an AI assistant capable of processing and summarizing complex documents with diagrams and tables." This instructs the AI to act as an assistant that can process and summarize complex documents.

`user_instruction` is a more specific instruction that tells the AI what to do. In your case, `user_prompt` is a detailed instruction asking the AI to analyze a document, provide a summary, and then extract key information and details from tables and diagrams in the form of bullet points.

Depending on the task, you can change these instructions. For example, if you want the AI to extract only the key points from a document without providing a summary, you can set `user_instruction` to "Please extract the key points from this document." If you want the AI to act as a translator, you can set `system_instruction` to "You are an AI assistant capable of translating text from English to French."

Remember, the instructions should be clear and specific to guide the AI to perform the desired task effectively.

In [19]:
from src.ocr.transformer import GPT4VisionManager

gpt_vision_client = GPT4VisionManager()
gpt_vision_client.load_environment_variables_from_env_file()

In [20]:
sys_message = "You are an AI assistant capable of processing and summarizing complex documents with diagrams and tables."
user_prompt = """
Please analyze this document and provide the information in the following format:

1. Summary: Provide a concise summary of the document, focusing on the main points and overall context.

2. Content: Include detailed, granular information extracted from the document, particularly from any tables and diagrams. This information should be presented in a structured format, such as a list or table, 
making sure all information is included and that the information is accurate and complete. 

3. Category: List key categories or keywords, with a focus on main products or concepts mentioned in the document. Categories should be abstracted and listed, separated by commas, with a maximum of 10 words.

The purpose is to enable another system to read and understand this information in detail, to facilitate answering precise questions based on the document's context.

Please return the information in the following format:

#summary
<summary text>

#content
<content text>

#category
[<category 1>, <category 2>, <category 3>, ...]
"""

## We want to abstract data from these two images 

In [ ]:
display(Image(image_file_path))

In [ ]:
image_file_paths = []

In [ ]:
ocr_recognizer = gpt_vision_client.call_gpt4v_image(
    image_file_path,
    system_instruction=sys_message,
    user_instruction=user_prompt,
    ocr=True,
    use_vision_api=True,
    display_image=True,
    max_tokens=2000,
    seed=42,
)